Loading up the survey data

In [ ]:
library(tidyverse)
reforms <- read_csv("Potential Reform Survey (Responses) - Form Responses 1.csv")

In [ ]:
reforms

In [ ]:
# Fixing names
names(reforms) <- gsub("How would you rate each of these reforms as good or bad for democracy\\?", "good", names(reforms))
names(reforms) <- gsub("How consequential would each of these reforms be\\?", "imp", names(reforms))
names(reforms) <- gsub("How feasible is each reform\\?", "feasible", names(reforms))
names(reforms) <- gsub("\\[Ranked choice voting for president\\]", "rcv", names(reforms))
names(reforms) <- gsub("\\[Abolish the electoral college for national popular vote\\]", "ec", names(reforms))
names(reforms) <- gsub("\\[House elected using proportional representation \\(by state\\)\\]", "pr", names(reforms))
names(reforms) <- gsub("\\[Election day a national holiday\\]", "hol", names(reforms))
names(reforms) <- gsub("\\[Ban partisan redistricting \\(gerrymandering\\)\\]", "gmd", names(reforms))
names(reforms) <- gsub("\\[18 year term-limit for Supreme Court justices\\]", "sc", names(reforms))
names(reforms) <- gsub("\\[Make Senate proportional to population\\]", "sen", names(reforms))
names(reforms) <- gsub("\\[Automatic registration at 18\\]", "reg", names(reforms))
names(reforms) <- gsub(" ", "_", names(reforms))
names(reforms)

In [ ]:
# Recoding the good variables
recode_good_bad <- function(x) {
  recode_vals <- c(
    "Definitely bad" = -1,
    "Probably bad" = -0.5,
    "Neutral/Unsure" = 0,
    "Probably good" = 0.5,
    "Definitely good" = 1
  )
  out <- recode_vals[x]
  if (length(out) != length(x)) return(rep(NA_real_, length(x)))  # fallback for mismatches
  return(out)
}

# variables to recode
target_cols <- names(reforms)[grepl("^good_", names(reforms)) & sapply(reforms, is.character)]

# Doing the recoding
for (col in target_cols) {
  reforms[[col]] <- recode_good_bad(reforms[[col]])
}


In [ ]:
# Recoding the importance variables
recode_imp <- function(x) {
  recode_vals <- c(
    "No impact" = 0,
    "Minimal impact" = .25,
    "Moderate impact" = .5,
    "Large impact" = 0.75,
    "Very large impact" = 1
  )
  out <- recode_vals[x]
  if (length(out) != length(x)) return(rep(NA_real_, length(x)))  # fallback for mismatches
  return(out)
}

# Which ones to recode
target_cols <- names(reforms)[grepl("^imp", names(reforms)) & sapply(reforms, is.character)]

# Doing the recoding
for (col in target_cols) {
  reforms[[col]] <- recode_imp(reforms[[col]])
}


In [ ]:
# Recoding the feasible variables
recode_feas <- function(x) {
  recode_vals <- c(
    "Impossible" = 0,
    "Very unlikely" = .25,
    "Somewhat unlikely" = .5,
    "Plausible" = 0.75,
    "Very plausible" = 1
  )
  out <- recode_vals[x]
  if (length(out) != length(x)) return(rep(NA_real_, length(x)))  # fallback for mismatches
  return(out)
}

# Which ones to recode
target_cols <- names(reforms)[grepl("^feasible", names(reforms)) & sapply(reforms, is.character)]

# Doing the recoding
for (col in target_cols) {
  reforms[[col]] <- recode_feas(reforms[[col]])
}

In [ ]:
head(reforms)

In [ ]:
# Now create an average for each of the variables
reform_keys <- c("pr", "rcv", "ec", "hol", "sen", "gmd", "sc", "reg")

# Initialize a new data frame
byref <- data.frame(
  reform = reform_keys,
  good = sapply(reform_keys, function(k) mean(reforms[[paste0("good_", k)]], na.rm = TRUE)),
  imp = sapply(reform_keys, function(k) mean(reforms[[paste0("imp_", k)]], na.rm = TRUE)),
  feasible = sapply(reform_keys, function(k) mean(reforms[[paste0("feasible_", k)]], na.rm = TRUE))
)

print(byref)


In [ ]:
plot(byref$feasible, byref$good, xlim=c(0,1), ylim=c(-1,1))
text(byref$feasible, byref$good-.05, byref$reform)